In [2]:
#imports
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import torch
from torch.utils.data import Dataset
from transformers import BartForConditionalGeneration, BartTokenizer
from datasets import load_dataset
from tqdm import tqdm

# import the metrics
from rouge_score import rouge_scorer
from bert_score import BERTScorer
#from evaluation.readability import flesch_kincaid_grade_level, dale_chall_readability_score, coleman_liau_index, lens

import json
import numpy as np
from samsum_dataset.preprocessing import remove_oov_words, clean_dataset, wsd

In [3]:
def _load_dataset() -> Dataset:
    from datasets import load_dataset
    dataset = load_dataset("Samsung/samsum", "samsum", trust_remote_code=True)
    
    #shuffle the train and validation sets
    #dataset['train'] = dataset['train'].shuffle()
    #dataset['validation'] = dataset['validation'].shuffle()
    
    return dataset

In [4]:
dataset1 = _load_dataset()
dataset2 = _load_dataset()
dataset3 = _load_dataset()

In [11]:
dataset2 = clean_dataset(dataset2)
dataset2 = remove_oov_words(dataset2)
# from samsum_dataset.preprocessing import expand_contractions
# def preprocess(entry):
#     entry['dialogue'] = expand_contractions(entry['dialogue'])
#     return entry

# dataset2['test'] = dataset2['test'].map(preprocess)

Building vocabulary:   0%|          | 0/14732 [00:00<?, ?it/s]

Building vocabulary: 100%|██████████| 14732/14732 [00:01<00:00, 9330.76it/s] 


In [12]:
dataset2['test'][0]['dialogue']

'hannah hey do you have betty s number amanda let me check hannah file_gif amanda sorry cannot find it amanda ask larry amanda he called her last time we were at the park together hannah i do not know him well hannah file_gif amanda do not be shy he is very nice hannah if you say so hannah i would rather you texted him amanda just text him hannah urgh alright hannah bye amanda bye bye'

In [8]:
#compare the two datasets
for split in ['train', 'validation', 'test']:
    print("Testing the dataset for split", split)
    
    for i in tqdm(range(len(dataset1[split]))):
        if dataset1[split][i]['summary'] == dataset2[split][i]['summary']:
            print(f"The summary for the {split} dataset is same for the entry {i}.")
            
        if dataset1[split][i]['dialogue'] == dataset2[split][i]['dialogue']:
            print(f"The dialogue for the {split} dataset is same for the entry {i}.")

Testing the dataset for split train


 18%|█▊        | 2595/14732 [00:00<00:02, 4308.96it/s]

The summary for the train dataset is same for the entry 1760.


 38%|███▊      | 5633/14732 [00:01<00:02, 4290.24it/s]

The summary for the train dataset is same for the entry 4889.


...

100%|██████████| 14732/14732 [00:03<00:00, 4256.74it/s]


Testing the dataset for split validation


100%|██████████| 818/818 [00:00<00:00, 4421.69it/s]


Testing the dataset for split test


100%|██████████| 819/819 [00:00<00:00, 3800.02it/s]


In [ ]:
for split in ['validation', 'test']:
    print("Testing the dataset for split", split)
    
    for i in tqdm(range(len(dataset2[split]))):
        #check if any of the splits contain the word 'unk'
        if 'unk' in dataset2[split][i]['summary'] or 'unk' in dataset2[split][i]['dialogue']:
            print(f"The summary for the {split} dataset contains the word 'unk' for the entry {i}.")

In [5]:
dataset3 = wsd(dataset3, ['test'])

Warming up PyWSD (takes ~10 secs)... took 5.26047420501709 secs.


Map:   0%|          | 0/819 [00:00<?, ? examples/s]

In [8]:
dataset3['test'][0]['dialogue']

"Hannah : Hey , do you have Betty 's phone_number.n.01 ? Amanda : lunar_excursion_module.n.01 me determine.v.08 Hannah : < file_gif > Amanda : regretful.a.01 , california.n.01 n't find.v.05 it . Amanda : ask.v.04 Larry Amanda : He shout.v.02 her last.a.02 time.n.06 we were at the parking_lot.n.01 together.r.04 Hannah : I do n't know.v.07 him well.r.09 Hannah : < file_gif > Amanda : Do n't be diffident.a.02 , he 's very nice.s.03 Hannah : If you say.v.10 so.. Hannah : I 'd rather.r.02 you textbook.n.01 him Amanda : Just textbook.n.01 him 🙂 Hannah : Urgh.. very_well.r.02 Hannah : adieu.n.01 Amanda : adieu.n.01 adieu.n.01"